<p>
This Jupyter notebook investigates a stock market trading algorithm that is described in the
(short) book <i>The 12% Solution: Earn A 12% Average Annual Return On Your Money, Beating
The S&P 500</i> by David Alan Carter.  According to the book this algorithm, which is
 referred to as "ETF rotation", yields a 12% return, on average.
</p>
<p>
In this notebook I attempt to reproduce the results described in <i>The 12% solution</i>.
This will verify the results reported in the book and validate the implementation.
</p>
<p>
<i>The 12% solution</i> is targeted at a readership that is not using analytical tools
like Python.  Following the proposed algorithm the reader trades one of four equity ETFs,
two bond ETFs or a cash ETF. The analysis of past ETF performance can be done using
on-line tools.
</p>
<p>
If the results reported in <i>The 12% solution</i> can be reproduced, it may be possible to
improve on these results.
</p>
<ul>
<li>
Expanding the universe of ETFs could improve performance since there may be ETFs that have
better performance in the previous quarter than the four equity ETFs.
</li>
<li>
The algorithm outlines in <i>The 12% solution</i> may be a momentum algorithm.  Instead of
picking the ETF that has the highest return over a three month period, it would be
interesting to examine ETFs that have increased each month over the three month period
(e.g., ETFs that have momentum).
</li>
<li>
The performance of the equity ETFs is compared to the "cash" ETF SHY.  If their
performance is worse, the the portfolio is rebalanced into SHY. Investigating
rebalancing into a bond ETF if it's performance is better than SHY and the equity
ETFs should be looked at.
</li>
<li>
The <i>The 12% solution</i> proposes a 60% equity/40% bond allocation in the portfolio.
This reduces volatility, but it also reduces return.  Using an all equity allocation
that rebalances into cash with ETFs have worse than cash performance is something
that should be looked at.
</li>
</ul>

<h2>
ETF Rotation Set
</h2>
<h3>
Equity ETFs
</h3>
<p>
In <i>The 12% Solution</i> there are four ETFs that are used in rotation, two bond funds
and one ETF (SHY) that is used as a proxy for "cash" (e.g., an asset at the risk free rate).
</p>
<ul>
<li>
<p>
IWM: iShares Russell 2000 ETF
</p>
<p>
IWM tracks a market-cap-weighted index of US small-cap stocks. The index selects
stocks ranked 1,001-3,000 by market cap.
</p>
</li>
<li>
<p>
MDY: SPDR S&P Midcap 400 ETF Trust
</p>
<p>
MDY tracks a market-cap-weighted index of midcap US companies.
</p>
</li>
<li>
<p>
QQQ: Invesco QQQ Trust
</p>
<p>
QQQ tracks a modified-market-cap-weighted index of 100 NASDAQ-listed stocks.
</p>
</li>
<li>
<p>
SPY: SPDR S&P 500 ETF Trust
</p>
<p>
SPY tracks a market-cap-weighted index of US large- and midcap stocks selected by
the S&P Committee (e.g., S&P 500).
</p>
</li>
</ul>
<h3>
Bond ETFs
</h3>
<ul>
<li>
<p>
TLT: iShares 20+ Year Treasury Bond ETF
</p>
<p>
TLT tracks a market-weighted index of debt issued by the US Treasury
with remaining maturities of 20 years or more.
</p>
</li>
<li>
<p>
JNK: SPDR Bloomberg High Yield Bond ETF
</p>
<p>
JNK tracks a market-weighted index of highly liquid, high-yield, US
dollar-denominated corporate bonds.
</p>
</li>
</ul>
<h3>
Cash Proxy
</h3>
<ul>
<li>
<p>
SHY: iShares 10-20 Year Treasury Bond ETF
</p>
<p>
SHY tracks a market weighted index of debt issued by the US Treasury
with 1-3 years remaining to maturity. Treasury STRIPS are excluded.
</p>
</li>
</ul>

In [6]:
from datetime import datetime, timedelta

import matplotlib
from tabulate import tabulate
from typing import List, Tuple
from pandas_datareader import data
import pypfopt as pyopt
from pypfopt import expected_returns
from pypfopt import risk_models
from pypfopt import plotting, CLA
import matplotlib.pyplot as plt
import scipy.stats as stats
import pandas as pd
import numpy as np
from pathlib import Path
import tempfile
import quantstats as qs

def get_market_data(file_name: str,
                    data_col: str,
                    symbols: List,
                    data_source: str,
                    start_date: datetime,
                    end_date: datetime) -> pd.DataFrame:
    """
      file_name: the file name in the temp directory that will be used to store the data
      data_col: the type of data - 'Adj Close', 'Close', 'High', 'Low', 'Open', Volume'
      symbols: a list of symbols to fetch data for
      data_source: yahoo, etc...
      start_date: the start date for the time series
      end_date: the end data for the time series
      Returns: a Pandas DataFrame containing the data.

      If a file of market data does not already exist in the temporary directory, fetch it from the
      data_source.
    """
    temp_root: str = tempfile.gettempdir() + '/'
    file_path: str = temp_root + file_name
    temp_file_path = Path(file_path)
    file_size = 0
    if temp_file_path.exists():
        file_size = temp_file_path.stat().st_size

    if file_size > 0:
        close_data = pd.read_csv(file_path, index_col='Date')
    else:
        panel_data: pd.DataFrame = data.DataReader(symbols, data_source, start_date, end_date)
        close_data: pd.DataFrame = panel_data[data_col]
        close_data.to_csv(file_path)
    assert len(close_data) > 0, f'Error reading data for {symbols}'
    return close_data


plt.style.use('seaborn-whitegrid')

equity_etfs = ['IWM', 'MDY', 'QQQ', 'SPY']
bond_etfs = ['JNK', 'TLT']
cash_etf = 'SHY'
etf_set = [*equity_etfs, *bond_etfs, cash_etf]

data_source = 'yahoo'
# The start date is the date used in the examples in The 12% Solution
# yyyy-mm-dd
start_date_str = '2008-01-03'
start_date: datetime = datetime.fromisoformat(start_date_str)
end_date: datetime = datetime.today() - timedelta(days=1)

twelve_percent_etf_file = 'twelve_percent_etf_close'

etf_set_close = get_market_data(file_name=twelve_percent_etf_file,
                                data_col='Close',
                                symbols=etf_set,
                                data_source=data_source,
                                start_date=start_date,
                                end_date=end_date)

equity_set_close = etf_set_close[equity_etfs]
corr_mat = round(equity_set_close.corr(), 3)

<h3>
ETF Correlation
</h3>
<p>
In <i>The 12% Solution</i> ETF rotation the ETF with the highest return in the previous
three months is selected, unless the return is less than the return of SHY, in which case
SHY is selected.
</p>
<p>
One possible problem with this rotation algorith is that the ETFs are highly
correlated. One ETF may do somewhat better than the other ETFs, but they are all
market ETFs and are likely to have similar performance.  If the "market" is down
then all of the ETFs will probably be down.  The ETFs are also likely to have
similar returns.
</p>
<p>
The correlation matrix below shows the correlation between the equity ETFs.
</p>

In [7]:
print(tabulate(corr_mat, headers=[*corr_mat.columns], tablefmt='fancy_grid'))

╒═════╤═══════╤═══════╤═══════╤═══════╕
│     │   IWM │   MDY │   QQQ │   SPY │
╞═════╪═══════╪═══════╪═══════╪═══════╡
│ IWM │ 1     │ 0.997 │ 0.943 │ 0.98  │
├─────┼───────┼───────┼───────┼───────┤
│ MDY │ 0.997 │ 1     │ 0.941 │ 0.983 │
├─────┼───────┼───────┼───────┼───────┤
│ QQQ │ 0.943 │ 0.941 │ 1     │ 0.983 │
├─────┼───────┼───────┼───────┼───────┤
│ SPY │ 0.98  │ 0.983 │ 0.983 │ 1     │
╘═════╧═══════╧═══════╧═══════╧═══════╛


<p>
The high correlation between the ETFs suggests two areas that are worth investigating:
</p>
<ul>
<li>
<p>
Would the financial performance be similar if only a single ETF were used. For example,
SPY?
</p>
</li>
<li>
<p>
If a larger ETF universe is used ETFs with lower correlation might be selected
resulting in better performance.
</p>
</li>
</ul>

In [ ]:
holdings = 100000
equity = 0.6
bonds = 0.4

trading_days = 253
days_in_quarter = trading_days // 4
days_in_month = trading_days // 12
